In [ ]:
from constants import CRS
import pandas as pd
import geopandas as gpd
import os
from tqdm import tqdm

In [ ]:
# census level
geo_level = 'block'

# demos filepath
DEMOS_PATH = 'data/nhgis/demos/nhgis0002_ds248_2020_block.csv'

# GIS directory
GIS_DIR = 'data/nhgis/gis'

# encoding
ENCODING = 'iso-8859-1'

# GIS key
KEY = 'GISJOIN'

# Population Column from Source Data
POP_ID = 'U7B001'

In [ ]:

# get shapefile from directory
shapefiles = []

for subdir, dirs, files in os.walk(GIS_DIR):
    for file in files:
        if file[-3:] == 'shp':
            shapefiles.append(subdir + '/' + file)

In [ ]:
read_shapes = lambda x : gpd.read_file(x)[[KEY, 'geometry']].set_index(KEY)

geometries = pd.concat(
    [read_shapes(filepath) for filepath in tqdm(shapefiles)]
)

In [ ]:
populations = pd.read_csv(DEMOS_PATH , usecols = [KEY, POP_ID], low_memory = True, encoding=ENCODING)

In [ ]:
merged = geometries.merge(populations, on=KEY)

In [ ]:
import dask

In [ ]:
merged.to_file('cache/nhgis.shp')

In [ ]:
geo_list = []

for file in shapefiles:
    geo = dask.delayed(read_shapes)(file)
    geo_list.append(geo)

total = dask.delayed(pd.concat)(geo_list)